# Code

## Data Cleaning

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from xgboost import XGBClassifier, XGBRegressor, DMatrix, cv
from sklearn.utils import resample

# Load dataset
file_path = 'defaultdata.csv'
data = pd.read_csv(file_path, header=1)

# Rename columns
column_mapping = {
    'LIMIT_BAL': 'Credit_Limit',
    'SEX': 'Gender',
    'EDUCATION': 'Education_Level',
    'MARRIAGE': 'Marital_Status',
    'AGE': 'Age',
    'PAY_0': 'Repayment_Status_1',
    'PAY_2': 'Repayment_Status_2',
    'PAY_3': 'Repayment_Status_3',
    'PAY_4': 'Repayment_Status_4',
    'PAY_5': 'Repayment_Status_5',
    'PAY_6': 'Repayment_Status_6',
    'BILL_AMT1': 'Bill_Amount_1',
    'BILL_AMT2': 'Bill_Amount_2',
    'BILL_AMT3': 'Bill_Amount_3',
    'BILL_AMT4': 'Bill_Amount_4',
    'BILL_AMT5': 'Bill_Amount_5',
    'BILL_AMT6': 'Bill_Amount_6',
    'PAY_AMT1': 'Payment_Amount_1',
    'PAY_AMT2': 'Payment_Amount_2',
    'PAY_AMT3': 'Payment_Amount_3',
    'PAY_AMT4': 'Payment_Amount_4',
    'PAY_AMT5': 'Payment_Amount_5',
    'PAY_AMT6': 'Payment_Amount_6',
    'default payment next month': 'Default_Payment'
}
data.rename(columns=column_mapping, inplace=True)

# Clean data
data = data[(data['Marital_Status'] != 0) & data['Education_Level'].isin([1, 2, 3])]
pay_features = ['Repayment_Status_1', 'Repayment_Status_2', 'Repayment_Status_3', 
                'Repayment_Status_4', 'Repayment_Status_5', 'Repayment_Status_6']
for feature in pay_features:
    data.loc[data[feature] < 0, feature] = -1
    data.loc[data[feature] >= 0, feature] += 1
    data[feature] = data[feature].astype('int64')

# Encode features
data['Grad_School'] = (data['Education_Level'] == 1).astype('int')
data['University'] = (data['Education_Level'] == 2).astype('int')
data.drop('Education_Level', axis=1, inplace=True)
data['Female'] = (data['Gender'] == 2).astype('int')
data['Married'] = (data['Marital_Status'] == 1).astype('int')
data.drop(['Gender', 'Marital_Status'], axis=1, inplace=True)
data.drop('ID', axis=1, inplace=True)

## Summary Statistics

### Age

In [ ]:
# Summary statistics for Age
age_summary = data['Age'].describe(percentiles=[0.25, 0.5, 0.75])
print("Summary Statistics for Age:")
print(age_summary)

# Histogram and density plot for Age
plt.style.use('default')
plt.figure(figsize=(10, 6))
sns.histplot(data['Age'], bins=30, kde=True, stat='density', color='gray', edgecolor='black', alpha=0.8)
sns.kdeplot(data['Age'], color='black', linewidth=2)
plt.title('Age Distribution', fontsize=14, weight='bold')
plt.xlabel('Age', fontsize=12, labelpad=10)
plt.ylabel('Density', fontsize=12, labelpad=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Create Age categories
data['Middle_Aged'] = ((data['Age'] > 30) & (data['Age'] <= 50)).astype(int)
data['Older'] = (data['Age'] > 50).astype(int)

### Repayment Status

In [ ]:
# Bar plot of Repayment Statuses
repayment_status_vars = [
    'Repayment_Status_1', 'Repayment_Status_2', 'Repayment_Status_3',
    'Repayment_Status_4', 'Repayment_Status_5', 'Repayment_Status_6'
]

renamed_status_vars = {
    'Repayment_Status_1': 'Repayment Status 1',
    'Repayment_Status_2': 'Repayment Status 2',
    'Repayment_Status_3': 'Repayment Status 3',
    'Repayment_Status_4': 'Repayment Status 4',
    'Repayment_Status_5': 'Repayment Status 5',
    'Repayment_Status_6': 'Repayment Status 6'
}

repayment_status_counts = {renamed_status_vars[var]: data[var].value_counts(sort=False) for var in repayment_status_vars}
repayment_status_df = pd.DataFrame(repayment_status_counts).fillna(0)
colors = ['#D3D3D3', '#A9A9A9', '#808080', '#696969', '#505050', '#303030']
ax = repayment_status_df.plot(kind='bar', figsize=(12, 8), width=0.8, color=colors, edgecolor='black')
plt.title('Distribution of Repayment Status Across All Periods', fontsize=14, weight='bold')
plt.xlabel('Repayment Status', fontsize=12, labelpad=10)
plt.ylabel('Count', fontsize=12, labelpad=10)
plt.legend(title='Period', loc='upper right', fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('Repaybar.png', dpi=600) 
plt.show()

# Create Repayment Status categories
data['Mild_Delay'] = ((data[repayment_status_vars] == 1).any(axis=1) & 
                      (data[repayment_status_vars] < 2).all(axis=1)).astype(int)
data['Severe_Delay'] = (data[repayment_status_vars] >= 2).any(axis=1).astype(int)
data = data[(data['Mild_Delay'] + data['Severe_Delay']) <= 1]

### Credit Limit

In [ ]:
# Summary statistics for Credit Limit
credit_limit_summary = data['Credit_Limit'].describe(percentiles=[0.25, 0.5, 0.75])
print("Summary Statistics for Credit Limit:")
print(credit_limit_summary)

# Histogram and density plot for Credit Limit
plt.style.use('default')
plt.figure(figsize=(10, 6))
sns.histplot(data['Credit_Limit'], bins=30, kde=True, stat='density', color='gray', edgecolor='black', alpha=0.8)
sns.kdeplot(data['Credit_Limit'], color='black', linewidth=2)
plt.title('Credit Limit Distribution', fontsize=14, weight='bold')
plt.xlabel('Credit Limit', fontsize=12, labelpad=10)
plt.ylabel('Density', fontsize=12, labelpad=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Create Credit Limit categories
data['Medium_Credit_Limit'] = ((data['Credit_Limit'] > 50000) & (data['Credit_Limit'] <= 240000)).astype(int)
data['High_Credit_Limit'] = (data['Credit_Limit'] > 240000).astype(int)

## Key Interactions

### Create Interactions

In [ ]:
# Interaction terms for Repayment Status × Age
data['Mild_Delay × Middle_Aged'] = data['Mild_Delay'] * data['Middle_Aged']
data['Mild_Delay × Older'] = data['Mild_Delay'] * data['Older']
data['Severe_Delay × Middle_Aged'] = data['Severe_Delay'] * data['Middle_Aged']
data['Severe_Delay × Older'] = data['Severe_Delay'] * data['Older']

# Interaction terms for Repayment Status × Credit Limit 
data['Medium_Credit_Limit × Mild_Delay'] = data['Medium_Credit_Limit'] * data['Mild_Delay']
data['Medium_Credit_Limit × Severe_Delay'] = data['Medium_Credit_Limit'] * data['Severe_Delay']
data['High_Credit_Limit × Mild_Delay'] = data['High_Credit_Limit'] * data['Mild_Delay']
data['High_Credit_Limit × Severe_Delay'] = data['High_Credit_Limit'] * data['Severe_Delay']

# Interaction terms for Education × Marital Status
data['Grad_School × Married'] = data['Grad_School'] * data['Married']
data['University × Married'] = data['University'] * data['Married']

### Interaction Statistics

In [ ]:
# Define combinations
repayment_age_combinations = {
    'Younger & On-Time': (data['Middle_Aged'] == 0) & (data['Older'] == 0) & 
                         (data['Mild_Delay'] == 0) & (data['Severe_Delay'] == 0),
    'Middle_Aged & On-Time': (data['Middle_Aged'] == 1) & (data['Older'] == 0) & 
                              (data['Mild_Delay'] == 0) & (data['Severe_Delay'] == 0),
    'Older & On-Time': (data['Older'] == 1) & 
                       (data['Mild_Delay'] == 0) & (data['Severe_Delay'] == 0),
    'Younger & Mild_Delay': (data['Middle_Aged'] == 0) & (data['Older'] == 0) & 
                            (data['Mild_Delay'] == 1) & (data['Severe_Delay'] == 0),
    'Middle_Aged & Mild_Delay': (data['Middle_Aged'] == 1) & (data['Older'] == 0) & 
                                 (data['Mild_Delay'] == 1) & (data['Severe_Delay'] == 0),
    'Older & Mild_Delay': (data['Older'] == 1) & 
                          (data['Mild_Delay'] == 1) & (data['Severe_Delay'] == 0),
    'Younger & Severe_Delay': (data['Middle_Aged'] == 0) & (data['Older'] == 0) & 
                              (data['Mild_Delay'] == 0) & (data['Severe_Delay'] == 1),
    'Middle_Aged & Severe_Delay': (data['Middle_Aged'] == 1) & (data['Older'] == 0) & 
                                   (data['Mild_Delay'] == 0) & (data['Severe_Delay'] == 1),
    'Older & Severe_Delay': (data['Older'] == 1) & 
                            (data['Mild_Delay'] == 0) & (data['Severe_Delay'] == 1)
}

credit_limit_repayment_combinations = {
    'Low Credit & On-Time': (data['Medium_Credit_Limit'] == 0) & (data['High_Credit_Limit'] == 0) & 
                            (data['Mild_Delay'] == 0) & (data['Severe_Delay'] == 0),
    'Medium Credit & On-Time': (data['Medium_Credit_Limit'] == 1) & (data['Mild_Delay'] == 0) & 
                               (data['Severe_Delay'] == 0),
    'High Credit & On-Time': (data['High_Credit_Limit'] == 1) & (data['Mild_Delay'] == 0) & 
                             (data['Severe_Delay'] == 0),
    'Low Credit & Mild_Delay': (data['Medium_Credit_Limit'] == 0) & (data['High_Credit_Limit'] == 0) & 
                               (data['Mild_Delay'] == 1),
    'Medium Credit & Mild_Delay': (data['Medium_Credit_Limit'] == 1) & (data['Mild_Delay'] == 1),
    'High Credit & Mild_Delay': (data['High_Credit_Limit'] == 1) & (data['Mild_Delay'] == 1),
    'Low Credit & Severe_Delay': (data['Medium_Credit_Limit'] == 0) & (data['High_Credit_Limit'] == 0) & 
                                 (data['Severe_Delay'] == 1),
    'Medium Credit & Severe_Delay': (data['Medium_Credit_Limit'] == 1) & (data['Severe_Delay'] == 1),
    'High Credit & Severe_Delay': (data['High_Credit_Limit'] == 1) & (data['Severe_Delay'] == 1)
}

education_marital_combinations = {
    'High School & Single': (data['Grad_School'] == 0) & (data['University'] == 0) & (data['Married'] == 0),
    'Grad School & Single': (data['Grad_School'] == 1) & (data['Married'] == 0),
    'University & Single': (data['University'] == 1) & (data['Married'] == 0),
    'High School & Married': (data['Grad_School'] == 0) & (data['University'] == 0) & (data['Married'] == 1),
    'Grad School & Married': (data['Grad_School'] == 1) & (data['Married'] == 1),
    'University & Married': (data['University'] == 1) & (data['Married'] == 1)
}

# Summary table generation
def calculate_combinations(combinations, total_count):
    stats = {k: v.sum() for k, v in combinations.items()}
    percentages = {k: (v / total_count) * 100 for k, v in stats.items()}
    return pd.DataFrame({'Count': stats, 'Percentage (%)': percentages})

total_count = len(data)
repayment_age_table = calculate_combinations(repayment_age_combinations, total_count)
credit_limit_repayment_table = calculate_combinations(credit_limit_repayment_combinations, total_count)
education_marital_table = calculate_combinations(education_marital_combinations, total_count)

print("Repayment Status × Age Combinations:")
print(repayment_age_table)
print("\nCredit Limit × Repayment Status Combinations:")
print(credit_limit_repayment_table)
print("\nEducation × Marital Status Combinations:")
print(education_marital_table)

# Default vs. Non-Default proportions
def calculate_default_proportions(combinations):
    proportions = {}
    for label, condition in combinations.items():
        subset = data[condition]
        default_counts = subset['Default_Payment'].value_counts(normalize=True) * 100
        proportions[label] = default_counts.reindex([0, 1], fill_value=0)
    return pd.DataFrame(proportions).T.rename(columns={0: 'Non-Default (%)', 1: 'Default (%)'})

repayment_age_default_proportions = calculate_default_proportions(repayment_age_combinations)
credit_limit_repayment_default_proportions = calculate_default_proportions(credit_limit_repayment_combinations)
education_marital_default_proportions = calculate_default_proportions(education_marital_combinations)

print("\nProportion of Default vs Non-Default for Repayment Status × Age Combinations:")
print(repayment_age_default_proportions)
print("\nProportion of Default vs Non-Default for Credit Limit × Repayment Status Combinations:")
print(credit_limit_repayment_default_proportions)
print("\nProportion of Default vs Non-Default for Education × Marital Status Combinations:")
print(education_marital_default_proportions)

### General Summary Statistics 

In [ ]:
# Calculate base cases
base_cases = {
    "Young (<30)": len(data.query("Middle_Aged == 0 & Older == 0")),
    "On Time Payment": len(data.query("Mild_Delay == 0 & Severe_Delay == 0")),
    "Low Credit Limit": len(data.query("Medium_Credit_Limit == 0 & High_Credit_Limit == 0")),
    "High School": len(data[(data["Grad_School"] == 0) & (data["University"] == 0)]),
    "Not Married": data["Married"].value_counts().get(0, 0),
}

# Variables of interest
categorical_vars = [
    "Default_Payment", "Grad_School", "University", "Female", "Married", 
    "Middle_Aged", "Older", "Mild_Delay", "Severe_Delay", 
    "Medium_Credit_Limit", "High_Credit_Limit"
]

# Initialize summary dictionary
summary = {"Variable": [], "Category": [], "Count": [], "% of Sample": []}

# Summarize categorical variables
for var in categorical_vars:
    for category in data[var].unique():
        count = len(data[data[var] == category])
        pct = (count / len(data)) * 100
        summary["Variable"].append(var)
        summary["Category"].append(f"{var} = {category}")
        summary["Count"].append(count)
        summary["% of Sample"].append(round(pct, 2))

# Add base cases
for base_case, count in base_cases.items():
    summary["Variable"].append(base_case)
    summary["Category"].append("Base Case")
    summary["Count"].append(count)
    summary["% of Sample"].append(round((count / len(data)) * 100, 2))

# Add stats for Default_Payment
default_1_count = len(data[data["Default_Payment"] == 1])
default_0_count = len(data[data["Default_Payment"] == 0])
summary["Variable"].extend(["Default_Payment", "Default_Payment"])
summary["Category"].extend(["Default = 1", "Not Default = 0"])
summary["Count"].extend([default_1_count, default_0_count])
summary["% of Sample"].extend([
    round((default_1_count / len(data)) * 100, 2), 
    round((default_0_count / len(data)) * 100, 2)
])

# Convert to DataFrame and display
summary_df = pd.DataFrame(summary)
print(summary_df)

## X-Learners

### Data Preparation 

In [ ]:
# Remove specified interaction terms
interaction_terms_to_remove = [
    'Mild_Delay × Middle_Aged', 'Mild_Delay × Older', 'Severe_Delay × Middle_Aged', 
    'Severe_Delay × Older', 'Medium_Credit_Limit × Mild_Delay', 
    'Medium_Credit_Limit × Severe_Delay', 'High_Credit_Limit × Mild_Delay', 
    'High_Credit_Limit × Severe_Delay', 'Grad_School × Married', 
    'University × Married'
]
data = data.drop(columns=interaction_terms_to_remove, errors='ignore')

# Define features and target variable
X = data.drop(columns=['Default_Payment'])
y = data['Default_Payment']

### Repayment Status × Age Interaction

In [ ]:
# Define subgroups for Repayment Status × Age
interaction_groups = {
    'Younger & On-Time': (X['Middle_Aged'] == 0) & (X['Older'] == 0) & 
                         (X['Mild_Delay'] == 0) & (X['Severe_Delay'] == 0),
    'Middle_Aged & On-Time': (X['Middle_Aged'] == 1) & (X['Older'] == 0) & 
                              (X['Mild_Delay'] == 0) & (X['Severe_Delay'] == 0),
    'Older & On-Time': (X['Older'] == 1) & 
                       (X['Mild_Delay'] == 0) & (X['Severe_Delay'] == 0),
    'Younger & Mild_Delay': (X['Middle_Aged'] == 0) & (X['Older'] == 0) & 
                            (X['Mild_Delay'] == 1) & (X['Severe_Delay'] == 0),
    'Middle_Aged & Mild_Delay': (X['Middle_Aged'] == 1) & (X['Older'] == 0) & 
                                 (X['Mild_Delay'] == 1) & (X['Severe_Delay'] == 0),
    'Older & Mild_Delay': (X['Older'] == 1) & 
                          (X['Mild_Delay'] == 1) & (X['Severe_Delay'] == 0),
    'Younger & Severe_Delay': (X['Middle_Aged'] == 0) & (X['Older'] == 0) & 
                              (X['Mild_Delay'] == 0) & (X['Severe_Delay'] == 1),
    'Middle_Aged & Severe_Delay': (X['Middle_Aged'] == 1) & (X['Older'] == 0) & 
                                   (X['Mild_Delay'] == 0) & (X['Severe_Delay'] == 1),
    'Older & Severe_Delay': (X['Older'] == 1) & 
                            (X['Mild_Delay'] == 0) & (X['Severe_Delay'] == 1)
}

# Dictionary to store final CATE results and confidence intervals
cate_results = {}

# CV function
def xgb_cv_optimization(X, y, params, model_type, nfold=5, verbose_eval=False):
    dmatrix = DMatrix(data=X, label=y)
    
    # Determine metrics and stratification
    if model_type == 'classifier':
        metrics = 'auc'
        stratified = True
    else:
        metrics = 'rmse'
        stratified = False
    
    # Perform CV
    cv_results = cv(
        params=params,
        dtrain=dmatrix,
        num_boost_round=200,
        nfold=nfold,
        metrics=metrics,
        stratified=stratified,
        early_stopping_rounds=10,
        seed=42,
        verbose_eval=verbose_eval
    )
    return len(cv_results)

# Bootstrapping function
def bootstrap_cate(X, y, treat, num_bootstrap):
    bootstrap_cates = []
    for i in range(num_bootstrap):
        X_resampled, y_resampled, treat_resampled = resample(X, y, treat, random_state=i)
        
        # Fit models for bootstrap sample
        propensity_model.fit(X_resampled, treat_resampled)
        propensity_scores = propensity_model.predict_proba(X_resampled)[:, 1]
        
        treated_indices = treat_resampled == 1
        control_indices = treat_resampled == 0
        
        treated_model.fit(X_resampled[treated_indices], y_resampled[treated_indices])
        control_model.fit(X_resampled[control_indices], y_resampled[control_indices])
        
        treated_preds = treated_model.predict(X)
        control_preds = control_model.predict(X)
        treatment_effects = treated_preds - control_preds
        
        # Combine effects
        refined_treated_model.fit(X_resampled[treated_indices], treatment_effects[treated_indices])
        refined_control_model.fit(X_resampled[control_indices], treatment_effects[control_indices])
        
        refined_treated_preds = refined_treated_model.predict(X)
        refined_control_preds = refined_control_model.predict(X)
        
        cate_combined = (propensity_scores * refined_treated_preds +
                         (1 - propensity_scores) * refined_control_preds)
        bootstrap_cates.append(cate_combined.mean())
    return np.array(bootstrap_cates)

# Loop through each group
for group_name, condition in interaction_groups.items():
    print(f"Processing group: {group_name}")
    
    # Define treatment variable
    treat = condition.astype(int)
    
    # Step 1: Propensity Score Estimation with XGBClassifier
    propensity_params = {
        'objective': 'binary:logistic',
        'learning_rate': 0.1,
        'max_depth': 5,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'eval_metric': 'auc',
        'seed': 42
    }
    best_num_boost_rounds = xgb_cv_optimization(X, treat, propensity_params, model_type='classifier', verbose_eval=False)
    propensity_model = XGBClassifier(
        n_estimators=best_num_boost_rounds,
        random_state=42,
        **propensity_params
    )
    propensity_model.fit(X, treat)
    propensity_scores = propensity_model.predict_proba(X)[:, 1]
    
    # Step 2: Fit Conditional Outcome Models
    treated_indices = X[treat == 1].index
    control_indices = X[treat == 0].index
    
    treated_outcome = y.loc[treated_indices]
    control_outcome = y.loc[control_indices]
    
    treated_params = {
        'objective': 'reg:squarederror',
        'learning_rate': 0.1,
        'max_depth': 5,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'eval_metric': 'rmse',
        'seed': 42
    }
    best_num_boost_rounds_treated = xgb_cv_optimization(
        X.loc[treated_indices], treated_outcome, treated_params, model_type='regressor', verbose_eval=False
    )
    treated_model = XGBRegressor(
        n_estimators=best_num_boost_rounds_treated,
        random_state=42,
        **treated_params
    )
    treated_model.fit(X.loc[treated_indices], treated_outcome)
    
    best_num_boost_rounds_control = xgb_cv_optimization(
        X.loc[control_indices], control_outcome, treated_params, model_type='regressor', verbose_eval=False
    )
    control_model = XGBRegressor(
        n_estimators=best_num_boost_rounds_control,
        random_state=42,
        **treated_params
    )
    control_model.fit(X.loc[control_indices], control_outcome)
    
    treated_preds = treated_model.predict(X)
    control_preds = control_model.predict(X)
    treatment_effects = treated_preds - control_preds
    
    refined_treated_model = XGBRegressor(**treated_params)
    refined_control_model = XGBRegressor(**treated_params)
    
    refined_treated_model.fit(X[treat == 1], treatment_effects[treat == 1])
    refined_control_model.fit(X[treat == 0], treatment_effects[treat == 0])
    
    refined_treated_preds = refined_treated_model.predict(X)
    refined_control_preds = refined_control_model.predict(X)
    
    cate_combined = (propensity_scores * refined_treated_preds +
                     (1 - propensity_scores) * refined_control_preds)
    
    # Bootstrap CATE
    bootstrap_cates = bootstrap_cate(X, y, treat, num_bootstrap=1000)
    cate_mean = cate_combined.mean()
    cate_ci_lower = np.percentile(bootstrap_cates, 2.5)
    cate_ci_upper = np.percentile(bootstrap_cates, 97.5)
    
    # Store results
    cate_results[group_name] = {
        'CATE Mean': cate_mean,
        'CI Lower': cate_ci_lower,
        'CI Upper': cate_ci_upper
    }

# Convert results to a DataFrame
cate_df = pd.DataFrame.from_dict(cate_results, orient='index')
print(cate_df)

### Repayment Status × Credit Limit Interaction

In [ ]:
# Define subgroups for Credit Limit × Repayment Status
interaction_groups = {
    'Low Credit & On-Time': (X['Medium_Credit_Limit'] == 0) & (X['High_Credit_Limit'] == 0) & 
                            (X['Mild_Delay'] == 0) & (X['Severe_Delay'] == 0),
    'Medium Credit & On-Time': (X['Medium_Credit_Limit'] == 1) & (X['Mild_Delay'] == 0) & 
                               (X['Severe_Delay'] == 0),
    'High Credit & On-Time': (X['High_Credit_Limit'] == 1) & (X['Mild_Delay'] == 0) & 
                             (X['Severe_Delay'] == 0),
    'Low Credit & Mild_Delay': (X['Medium_Credit_Limit'] == 0) & (X['High_Credit_Limit'] == 0) & 
                               (X['Mild_Delay'] == 1),
    'Medium Credit & Mild_Delay': (X['Medium_Credit_Limit'] == 1) & (X['Mild_Delay'] == 1),
    'High Credit & Mild_Delay': (X['High_Credit_Limit'] == 1) & (X['Mild_Delay'] == 1),
    'Low Credit & Severe_Delay': (X['Medium_Credit_Limit'] == 0) & (X['High_Credit_Limit'] == 0) & 
                                 (X['Severe_Delay'] == 1),
    'Medium Credit & Severe_Delay': (X['Medium_Credit_Limit'] == 1) & (X['Severe_Delay'] == 1),
    'High Credit & Severe_Delay': (X['High_Credit_Limit'] == 1) & (X['Severe_Delay'] == 1)
}

# Dictionary to store final CATE results and confidence intervals
cate_results = {}

# CV function
def xgb_cv_optimization(X, y, params, model_type, nfold=5, verbose_eval=False):
    dmatrix = DMatrix(data=X, label=y)
    
    # Determine metrics and stratification
    if model_type == 'classifier':
        metrics = 'auc'
        stratified = True
    else:
        metrics = 'rmse'
        stratified = False
    
    # Perform CV
    cv_results = cv(
        params=params,
        dtrain=dmatrix,
        num_boost_round=200,
        nfold=nfold,
        metrics=metrics,
        stratified=stratified,
        early_stopping_rounds=10,
        seed=42,
        verbose_eval=verbose_eval
    )
    return len(cv_results)

# Bootstrapping function
def bootstrap_cate(X, y, treat, num_bootstrap):
    bootstrap_cates = []
    for i in range(num_bootstrap):
        X_resampled, y_resampled, treat_resampled = resample(X, y, treat, random_state=i)
        
        # Fit models for bootstrap sample
        propensity_model.fit(X_resampled, treat_resampled)
        propensity_scores = propensity_model.predict_proba(X_resampled)[:, 1]
        
        treated_indices = treat_resampled == 1
        control_indices = treat_resampled == 0
        
        treated_model.fit(X_resampled[treated_indices], y_resampled[treated_indices])
        control_model.fit(X_resampled[control_indices], y_resampled[control_indices])
        
        treated_preds = treated_model.predict(X)
        control_preds = control_model.predict(X)
        treatment_effects = treated_preds - control_preds
        
        # Combine effects
        refined_treated_model.fit(X_resampled[treated_indices], treatment_effects[treated_indices])
        refined_control_model.fit(X_resampled[control_indices], treatment_effects[control_indices])
        
        refined_treated_preds = refined_treated_model.predict(X)
        refined_control_preds = refined_control_model.predict(X)
        
        cate_combined = (propensity_scores * refined_treated_preds +
                         (1 - propensity_scores) * refined_control_preds)
        bootstrap_cates.append(cate_combined.mean())
    return np.array(bootstrap_cates)

# Loop through each group
for group_name, condition in interaction_groups.items():
    print(f"Processing group: {group_name}")
    
    # Define treatment variable
    treat = condition.astype(int)
    
    # Step 1: Propensity Score Estimation with XGBClassifier
    propensity_params = {
        'objective': 'binary:logistic',
        'learning_rate': 0.1,
        'max_depth': 5,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'eval_metric': 'auc',
        'seed': 42
    }
    best_num_boost_rounds = xgb_cv_optimization(X, treat, propensity_params, model_type='classifier', verbose_eval=False)
    propensity_model = XGBClassifier(
        n_estimators=best_num_boost_rounds,
        random_state=42,
        **propensity_params
    )
    propensity_model.fit(X, treat)
    propensity_scores = propensity_model.predict_proba(X)[:, 1]
    
    # Step 2: Fit Conditional Outcome Models
    treated_indices = X[treat == 1].index
    control_indices = X[treat == 0].index
    
    treated_outcome = y.loc[treated_indices]
    control_outcome = y.loc[control_indices]
    
    treated_params = {
        'objective': 'reg:squarederror',
        'learning_rate': 0.1,
        'max_depth': 5,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'eval_metric': 'rmse',
        'seed': 42
    }
    best_num_boost_rounds_treated = xgb_cv_optimization(
        X.loc[treated_indices], treated_outcome, treated_params, model_type='regressor', verbose_eval=False
    )
    treated_model = XGBRegressor(
        n_estimators=best_num_boost_rounds_treated,
        random_state=42,
        **treated_params
    )
    treated_model.fit(X.loc[treated_indices], treated_outcome)
    
    best_num_boost_rounds_control = xgb_cv_optimization(
        X.loc[control_indices], control_outcome, treated_params, model_type='regressor', verbose_eval=False
    )
    control_model = XGBRegressor(
        n_estimators=best_num_boost_rounds_control,
        random_state=42,
        **treated_params
    )
    control_model.fit(X.loc[control_indices], control_outcome)
    
    treated_preds = treated_model.predict(X)
    control_preds = control_model.predict(X)
    treatment_effects = treated_preds - control_preds
    
    refined_treated_model = XGBRegressor(**treated_params)
    refined_control_model = XGBRegressor(**treated_params)
    
    refined_treated_model.fit(X[treat == 1], treatment_effects[treat == 1])
    refined_control_model.fit(X[treat == 0], treatment_effects[treat == 0])
    
    refined_treated_preds = refined_treated_model.predict(X)
    refined_control_preds = refined_control_model.predict(X)
    
    cate_combined = (propensity_scores * refined_treated_preds +
                     (1 - propensity_scores) * refined_control_preds)
    
    # Bootstrap CATE
    bootstrap_cates = bootstrap_cate(X, y, treat, num_bootstrap=1000)
    cate_mean = cate_combined.mean()
    cate_ci_lower = np.percentile(bootstrap_cates, 2.5)
    cate_ci_upper = np.percentile(bootstrap_cates, 97.5)
    
    # Store results
    cate_results[group_name] = {
        'CATE Mean': cate_mean,
        'CI Lower': cate_ci_lower,
        'CI Upper': cate_ci_upper
    }

# Convert results to a DataFrame
cate_df = pd.DataFrame.from_dict(cate_results, orient='index')
print(cate_df)

### Education × Martial Status Interaction

In [ ]:
# Define subgroups for Education × Marital Status
interaction_groups = {
    'High School & Single': (X['Grad_School'] == 0) & (X['University'] == 0) & (X['Married'] == 0),
    'Grad School & Single': (X['Grad_School'] == 1) & (X['Married'] == 0),
    'University & Single': (X['University'] == 1) & (X['Married'] == 0),
    'High School & Married': (X['Grad_School'] == 0) & (X['University'] == 0) & (X['Married'] == 1),
    'Grad School & Married': (X['Grad_School'] == 1) & (X['Married'] == 1),
    'University & Married': (X['University'] == 1) & (X['Married'] == 1)
}

# Dictionary to store final CATE results and confidence intervals
cate_results = {}

# CV function
def xgb_cv_optimization(X, y, params, model_type, nfold=5, verbose_eval=False):
    dmatrix = DMatrix(data=X, label=y)
    
    # Determine metrics and stratification
    if model_type == 'classifier':
        metrics = 'auc'
        stratified = True
    else:
        metrics = 'rmse'
        stratified = False
    
    # Perform CV
    cv_results = cv(
        params=params,
        dtrain=dmatrix,
        num_boost_round=200,
        nfold=nfold,
        metrics=metrics,
        stratified=stratified,
        early_stopping_rounds=10,
        seed=42,
        verbose_eval=verbose_eval
    )
    return len(cv_results)

# Bootstrapping function
def bootstrap_cate(X, y, treat, num_bootstrap):
    bootstrap_cates = []
    for i in range(num_bootstrap):
        X_resampled, y_resampled, treat_resampled = resample(X, y, treat, random_state=i)
        
        # Fit models for bootstrap sample
        propensity_model.fit(X_resampled, treat_resampled)
        propensity_scores = propensity_model.predict_proba(X_resampled)[:, 1]
        
        treated_indices = treat_resampled == 1
        control_indices = treat_resampled == 0
        
        treated_model.fit(X_resampled[treated_indices], y_resampled[treated_indices])
        control_model.fit(X_resampled[control_indices], y_resampled[control_indices])
        
        treated_preds = treated_model.predict(X)
        control_preds = control_model.predict(X)
        treatment_effects = treated_preds - control_preds
        
        # Combine effects
        refined_treated_model.fit(X_resampled[treated_indices], treatment_effects[treated_indices])
        refined_control_model.fit(X_resampled[control_indices], treatment_effects[control_indices])
        
        refined_treated_preds = refined_treated_model.predict(X)
        refined_control_preds = refined_control_model.predict(X)
        
        cate_combined = (propensity_scores * refined_treated_preds +
                         (1 - propensity_scores) * refined_control_preds)
        bootstrap_cates.append(cate_combined.mean())
    return np.array(bootstrap_cates)

# Loop through each group
for group_name, condition in interaction_groups.items():
    print(f"Processing group: {group_name}")
    
    # Define treatment variable
    treat = condition.astype(int)
    
    # Step 1: Propensity Score Estimation with XGBClassifier
    propensity_params = {
        'objective': 'binary:logistic',
        'learning_rate': 0.1,
        'max_depth': 5,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'eval_metric': 'auc',
        'seed': 42
    }
    best_num_boost_rounds = xgb_cv_optimization(X, treat, propensity_params, model_type='classifier', verbose_eval=False)
    propensity_model = XGBClassifier(
        n_estimators=best_num_boost_rounds,
        random_state=42,
        **propensity_params
    )
    propensity_model.fit(X, treat)
    propensity_scores = propensity_model.predict_proba(X)[:, 1]
    
    # Step 2: Fit Conditional Outcome Models
    treated_indices = X[treat == 1].index
    control_indices = X[treat == 0].index
    
    treated_outcome = y.loc[treated_indices]
    control_outcome = y.loc[control_indices]
    
    treated_params = {
        'objective': 'reg:squarederror',
        'learning_rate': 0.1,
        'max_depth': 5,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'eval_metric': 'rmse',
        'seed': 42
    }
    best_num_boost_rounds_treated = xgb_cv_optimization(
        X.loc[treated_indices], treated_outcome, treated_params, model_type='regressor', verbose_eval=False
    )
    treated_model = XGBRegressor(
        n_estimators=best_num_boost_rounds_treated,
        random_state=42,
        **treated_params
    )
    treated_model.fit(X.loc[treated_indices], treated_outcome)
    
    best_num_boost_rounds_control = xgb_cv_optimization(
        X.loc[control_indices], control_outcome, treated_params, model_type='regressor', verbose_eval=False
    )
    control_model = XGBRegressor(
        n_estimators=best_num_boost_rounds_control,
        random_state=42,
        **treated_params
    )
    control_model.fit(X.loc[control_indices], control_outcome)
    
    treated_preds = treated_model.predict(X)
    control_preds = control_model.predict(X)
    treatment_effects = treated_preds - control_preds
    
    refined_treated_model = XGBRegressor(**treated_params)
    refined_control_model = XGBRegressor(**treated_params)
    
    refined_treated_model.fit(X[treat == 1], treatment_effects[treat == 1])
    refined_control_model.fit(X[treat == 0], treatment_effects[treat == 0])
    
    refined_treated_preds = refined_treated_model.predict(X)
    refined_control_preds = refined_control_model.predict(X)
    
    cate_combined = (propensity_scores * refined_treated_preds +
                     (1 - propensity_scores) * refined_control_preds)
    
    # Bootstrap CATE
    bootstrap_cates = bootstrap_cate(X, y, treat, num_bootstrap=1000)
    cate_mean = cate_combined.mean()
    cate_ci_lower = np.percentile(bootstrap_cates, 2.5)
    cate_ci_upper = np.percentile(bootstrap_cates, 97.5)
    
    # Store results
    cate_results[group_name] = {
        'CATE Mean': cate_mean,
        'CI Lower': cate_ci_lower,
        'CI Upper': cate_ci_upper
    }

# Convert results to a DataFrame
cate_df = pd.DataFrame.from_dict(cate_results, orient='index')
print(cate_df)